In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from explorer import Explorer

In [ ]:
# e = Explorer()
# e.adaptor.initialize_env()
# e.explore()

In [ ]:
e = Explorer()

In [ ]:
for i in range(20):
    e.adaptor.initialize_env()
    e.explore()

In [ ]:
# a = Explorer()
# a.adaptor.initialize_env()
# a.execute_step()